In [1]:
import geopandas as gpd
import momepy
import networkx as nx
import graph_tool.all as gt
from scripts.nx2gt import nx2gt
import pandas as pd
import matplotlib
import itertools
from shapely.geometry import LineString, point
from shapely.wkt import loads
from tqdm import tqdm
import numpy as np
from dask.distributed import Client, LocalCluster
from dask import delayed
import dask
import math
import concurrent.futures
import matplotlib.pyplot as plt
import osmnx as ox
from scripts.graph_custom import generate_ego_graph, closeness_centrality

In [ ]:
local_crs = 27700
place = "test"
lat = 55.86421405612109
lng = -4.251846930489373
country = "UK"
crs=4326
radius=1

In [ ]:
streets = gpd.read_parquet(f"./output/{place}/streets_raw.pq").to_crs(local_crs).reset_index(drop=True)

In [ ]:
streets

In [ ]:
daskCluster = LocalCluster(threads_per_worker=2,
                n_workers=8, memory_limit='100GB')

client = Client(daskCluster)

client

In [ ]:
# Create a pandas dataframe from the multigraph
primal = momepy.gdf_to_nx(streets, approach='primal')

In [ ]:
vertID_dict = {}

In [ ]:
counter = itertools.count()

for node in primal.nodes():
    id = next(counter)
    primal.nodes[node]['vertID'] = id
    vertID_dict[id] = node

In [ ]:
inverted_vertID_dict = {value: key for key, value in vertID_dict.items()}

# Street Network Shapes

In [ ]:
edg_lin = momepy.Linearity(streets)
streets['linearity'] = edg_lin.series

## Node Degree

In [ ]:
# Convert it to a graph-tool graph
gtG = nx2gt(primal)

gtG.list_properties()

g = gt.GraphView(gtG)
mm_len = gtG.edge_properties["mm_len"]
vp, ep = gt.betweenness(g, weight = mm_len)

node_degree = {g.vertex_index[v]: v.in_degree() + v.out_degree() for v in g.vertices()}
# convert ID_dict to a DataFrame and rename the index and column
node_degree = pd.DataFrame.from_dict(node_degree, orient='index').rename(columns={0: "node_degree"})
# merge vertID_df and vertices on vertID and node, respectively

In [ ]:
# # cross-testing to Momepy implementation

# degree = dict(nx.degree(primal))
# nx.set_node_attributes(primal, degree, 'degree')

# graph = momepy.node_degree(primal, name='degree')

# nodes, edges, sw = momepy.nx_to_gdf(graph, points=True, lines=True,
#                                     spatial_weights=True)

# f, ax = plt.subplots(figsize=(10, 10))

# # Plot edges with a lower zorder
# edges.plot(ax=ax, color='lightgrey', zorder=1)

# # Plot nodes with a higher zorder
# nodes.plot(ax=ax, column='degree', cmap='RdYlBu', markersize=3, zorder=2)

# # Plot nodes
# scatter = ax.scatter(nodes.geometry.x, nodes.geometry.y, c=nodes['degree'], cmap='RdYlBu', s=3, zorder=2)

# # Create a legend
# legend1 = ax.legend(*scatter.legend_elements(num=5),
#                     loc="upper right", title="Node Degree")
# ax.add_artist(legend1)

# ax.set_axis_off()
# plt.show()

# node_degree["node_degree"]
# node_degree_aligned = node_degree["node_degree"].reset_index(drop=True)
# nodes_aligned = nodes["degree"].reset_index(drop=True)

# # Check if all elements are equal
# if not (node_degree_aligned == nodes_aligned).all():
#     raise ValueError("All elements in node_degree_aligned and nodes_aligned are identical.")

# Meshedness + clustering

Eucledean Dist

In [ ]:
primal_scattered = client.scatter(primal)

@delayed
def compute_ego_graph_eucl(primal_scattered, n, radius, distance):
    graph = nx2gt(nx.ego_graph(primal_scattered, n, radius=radius, distance=distance))
    return (n, graph)

tasks = []
for n in primal.nodes():
    task = compute_ego_graph_eucl(primal_scattered, n, radius=500, distance="mm_len")
    tasks.append(task)

results = dask.compute(*tasks)

In [ ]:
def calculate_meshedness(graph):
    e = graph.num_edges()
    v = graph.num_vertices()

    if v < 3:  # To avoid division by zero in the formula
        return 0

    meshedness_coefficient = (e - v + 1) / (2 * v - 5)
    return meshedness_coefficient

In [ ]:
meshedness_eucl_500 = []

ego_graph_eucl_collection = {}

for n, ego_graph in results:
    
    meshedness_eucl_500.append(calculate_meshedness(ego_graph))
    
    if n in ego_graph_eucl_collection.keys():
        raise Exception("n in ego_graph_collection")
    
    ego_graph_eucl_collection[inverted_vertID_dict[n]] = {}
    # 
    ego_graph_eucl_collection[inverted_vertID_dict[n]]["graph"] = ego_graph

    if len(list(ego_graph.edges())) > 1:
        ego_graph_eucl_collection[inverted_vertID_dict[n]]["weight"] = ego_graph.edge_properties["mm_len"]
    else:
        ego_graph_eucl_collection[inverted_vertID_dict[n]]["weight"] = None
        
    ego_graph_eucl_collection[inverted_vertID_dict[n]]["node"] = ego_graph.get_vertices()[[i for i,v in enumerate(ego_graph.vp.vertID.get_array()) if v == inverted_vertID_dict[n]][0]]

In [ ]:
meshedness_eucl_500 = pd.Series(meshedness_eucl_500)

In [ ]:
# ## validation to networkx
# graph = momepy.gdf_to_nx(streets)

# graph = momepy.clustering(graph, name='clustering')

# graph = momepy.meshedness(graph, radius=500, name='meshedness_eucl_500', distance='mm_len')

# nodes = momepy.nx_to_gdf(graph, points=True, lines=False, spatial_weights=False)

# f, ax = plt.subplots(figsize=(10, 10))
# # Plot nodes with a higher zorder
# nodes.plot(ax=ax, column='clustering', cmap='RdYlBu', markersize=3, zorder=2)
# streets.plot(ax=ax, color='lightgrey', alpha=0.5, zorder=1)
# ax.set_axis_off()
# plt.show()

# meshedness_eucl_500 = meshedness_eucl_500.reset_index(drop=True)
# nodes_aligned = nodes["meshedness_eucl_500"].reset_index(drop=True)

# # Check if all elements are equal
# if not (meshedness_eucl_500 == nodes_aligned).all():
#     raise ValueError("All elements in node_degree_aligned and nodes_aligned are not identical.")

Topological Distance

In [ ]:
primal_scattered = client.scatter(primal)

@delayed
def compute_ego_graph_topo(primal_scattered, n, radius):
    graph = nx2gt(nx.ego_graph(primal_scattered, n, radius, distance=None))
    return (n, graph)

tasks = []
for n in primal.nodes():
    task = compute_ego_graph_topo(primal_scattered, n, 5)
    tasks.append(task)
    
results = dask.compute(*tasks)


In [ ]:
def calculate_meshedness(graph):
    e = graph.num_edges()
    v = graph.num_vertices()

    if v < 3:  # To avoid division by zero in the formula
        return 0

    meshedness_coefficient = (e - v + 1) / (2 * v - 5)
    return meshedness_coefficient

In [ ]:
ego_graph_top_collection = {}

In [ ]:
meshedness_topo_500 = []

ego_graph_topo_collection = {}

for n, ego_graph in results:
    
    meshedness_topo_500.append(calculate_meshedness(ego_graph))
    
    if n in ego_graph_topo_collection.keys():
        raise Exception("n in ego_graph_collection")
    
    ego_graph_topo_collection[inverted_vertID_dict[n]] = {}
    # 
    ego_graph_topo_collection[inverted_vertID_dict[n]]["graph"] = ego_graph

    if len(list(ego_graph.edges())) > 1:
        ego_graph_topo_collection[inverted_vertID_dict[n]]["weight"] = ego_graph.edge_properties["mm_len"]
    else:
        ego_graph_topo_collection[inverted_vertID_dict[n]]["weight"] = None
        
    ego_graph_topo_collection[inverted_vertID_dict[n]]["node"] = ego_graph.get_vertices()[[i for i,v in enumerate(ego_graph.vp.vertID.get_array()) if v == inverted_vertID_dict[n]][0]]

In [ ]:
meshedness_topo_500 = pd.Series(meshedness_topo_500)

In [ ]:
meshedness_topo_500

In [ ]:
# ## validation to networkx
# graph = momepy.gdf_to_nx(streets)

# graph = momepy.clustering(graph, name='clustering')

# graph = momepy.meshedness(graph, radius=5, name='meshedness_top_5')

# nodes = momepy.nx_to_gdf(graph, points=True, lines=False, spatial_weights=False)

# f, ax = plt.subplots(figsize=(10, 10))
# # Plot nodes with a higher zorder
# nodes.plot(ax=ax, column='clustering', cmap='RdYlBu', markersize=3, zorder=2)
# streets.plot(ax=ax, color='lightgrey', alpha=0.5, zorder=1)
# ax.set_axis_off()
# plt.show()

# meshedness_top_5 = meshedness_top_500.reset_index(drop=True)
# nodes_aligned = nodes["meshedness_top_5"].reset_index(drop=True)

# # Check if all elements are equal
# if not (meshedness_top_5 == nodes_aligned).all():
#     raise ValueError("All elements in node_degree_aligned and nodes_aligned are not identical.")

# Multiple Centrality Analysis

## Closeness Centrality

In [ ]:
import geopandas as gpd
import momepy
import osmnx as ox
import matplotlib.pyplot as pl

In [ ]:
# primal = momepy.closeness_centrality(primal, radius=400, name='closeness400', distance='mm_len', weight='mm_len')

In [ ]:
# # test with NetworkX implementation
# graph = momepy.closeness_centrality(primal, radius=500, name='closeness_centrality_eucl_500', distance='weight', weight='mm_len')

# nodes = momepy.nx_to_gdf(graph, points=True, lines=False, spatial_weights=False)

# nodes = momepy.nx_to_gdf(primal, lines=False)
# f, ax = plt.subplots(figsize=(15, 15))
# nodes.plot(ax=ax, column='closeness_centrality_eucl_500', cmap='RdYlBu', markersize=3, zorder=2)
# streets.plot(ax=ax, color='lightgrey', alpha=0.5, zorder=1)
# ax.set_axis_off()
# ax.set_title('closeness400')
# plt.show()


In [2]:
streets_graph = ox.graph_from_place('Callander', network_type='drive')
streets_graph = ox.projection.project_graph(streets_graph)

edges = ox.graph_to_gdfs(ox.get_undirected(streets_graph), nodes=False, edges=True,
                                   node_geometry=False, fill_edge_geometry=True)

primal = momepy.gdf_to_nx(edges, approach='primal')

In [3]:
ego_graph_eucl_collection = generate_ego_graph(primal, radius = 500, distance='weight')

100%|██████████| 148/148 [00:00<00:00, 10680.98it/s]

{(627038.5074345332,
  5117626.126329217): <networkx.classes.multigraph.MultiGraph at 0x7f4d6e478710>,
 (626864.732652552,
  5117223.882441361): <networkx.classes.multigraph.MultiGraph at 0x7f4d6e42a240>,
 (626752.7313780212,
  5117245.128275525): <networkx.classes.multigraph.MultiGraph at 0x7f4d6e495250>,
 (626623.001432891,
  5115386.002563549): <networkx.classes.multigraph.MultiGraph at 0x7f4d6e47a780>,
 (626768.7762333312,
  5117241.903403768): <networkx.classes.multigraph.MultiGraph at 0x7f4d6e47bc80>,
 (626515.7700281657,
  5117306.377084477): <networkx.classes.multigraph.MultiGraph at 0x7f4d6e4297c0>,
 (626952.0468675159,
  5117111.545272634): <networkx.classes.multigraph.MultiGraph at 0x7f4d6e4783e0>,
 (626963.3822764186,
  5117097.5855789445): <networkx.classes.multigraph.MultiGraph at 0x7f4d6e42b500>,
 (626888.0721792183,
  5117172.222479096): <networkx.classes.multigraph.MultiGraph at 0x7f4d6e478d40>,
 (626660.3273137062,
  5115387.796684885): <networkx.classes.multigraph.Mu

In [ ]:
momepy.closeness_centrality(primal, radius=500, name='closeness_centrality_eucl_500', distance='weight', weight='mm_len')

In [ ]:
streets_graph = ox.graph_from_place('Callander', network_type='drive')
streets_graph = ox.projection.project_graph(streets_graph)

edges = ox.graph_to_gdfs(ox.get_undirected(streets_graph), nodes=False, edges=True,
                                   node_geometry=False, fill_edge_geometry=True)

primal = momepy.gdf_to_nx(edges, approach='primal')

primal = cus_gra.closeness_centrality(primal, radius=400, name='closeness400', distance='mm_len', weight='mm_len')